In [ ]:
import sys
sys.path.append('../input/timm-3monthsold/pytorch-image-models-master 2')
sys.path.append('../input/convnext')
from timm import create_model
#from timm.data.mixup import Mixup
from fastai.vision.all import *
#from fastai.callback.hook import *
import models.convnext
import models.convnext_isotropic
import timm
from timm.models.registry import register_model
import utils
import fastai

print("Torch version:", torch.__version__)
print("Timm version:", timm.__version__)
print("Fast-ai version:", fastai.__version__)

In [ ]:
!pip install ../input/albumentation-file/albumentations-1.0.3-py3-none-any.whl
!pip install ../input/madgrad-file/madgrad-1.1-py3-none-any.whl

In [ ]:
import typing as tp
sys.path.append("../input/volo-package")
from volo.models import volo_d1, volo_d2, volo_d3, volo_d4, volo_d5  # register models to timm
from volo.utils import load_pretrained_weights as volo_load_weights

In [ ]:
from madgrad import MADGRAD
from radam import Over9000

def WrapperMADGRAD(param_groups,**kwargs):
    return OptimWrapper(param_groups,MADGRAD)

def WrapperOver9000(param_groups,**kwargs):
    return OptimWrapper(param_groups,Over9000)

In [ ]:
def seed_everything(seed=999):
    set_seed(seed, reproducible=True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

    
def petfinder_rmse(preds, target):
    return 100 * torch.sqrt(F.mse_loss(F.sigmoid(preds.flatten()), target))

    
seed_everything(seed=365)

# # Cant use internet, and timm library wants to find pretrained weights (downloads if not in specific directory)
#     # so we make the directory
# if not os.path.exists('/root/cache/torch/hub/checkpoints/'):
#     os.makedirs('/root/.cache/torch/hub/checkpoints/')
# !cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
# !cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class VoloModel(nn.Module):
    
    def __init__(self, 
                 base_name: str, 
                 dims_head: tp.List[int],
                 pretrained=True, 
                 in_channels: int=3, 
                 image_size: int=224,
                 drop_path_rate: float=0.1
    ):
        """Initialize"""
        self.base_name = base_name
        super().__init__()
        model_name = base_name.split("-")[0]
        assert timm.is_model(model_name), "you can use only models in timm."
        
        if model_name[:4] == "volo":
            base_model = timm.create_model(
                model_name, img_size=image_size,
                mix_token=False, 
                return_dense=False, 
                drop_path_rate=0.3, # 0.4,  0.1
            )
            in_features = base_model.head.in_features
            if pretrained:
                volo_load_weights(base_model, VOLO_CHECHPOINTS[base_name], strict=False)
            
            if in_channels != 3:
                # # change input channel
                # # I follow the manner used in timm.
                first_conv = base_model.patch_embed.conv[0]
                w_t = first_conv.weight.data  # shape: (out_ch, 3, 7, 7)
                if in_channels == 1:
                    new_w_t = w_t.sum(axis=1, keepdims=True)  # shape: (out_ch, 1, 7, 7)
                else:
                    n_repeats = (in_channels + 3 - 1) // 3
                    new_w_t = w_t.repeat((1, n_repeats, 1, 1))
                    new_w_t = new_w_t[:, :in_channels]
                    new_w_t = new_w_t * 3 / in_channels  # shape: (out_ch, in_channels, 7, 7)

                first_conv.weight.data = new_w_t
        else:
            base_model = timm.create_model(
                base_name, pretrained=pretrained, in_chans=in_channels)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)
            
        base_model.reset_classifier(num_classes=0)
        self.backbone = base_model
        print(f"{base_name}: {in_features}")
        
        # # prepare head clasifier
        if dims_head[0] is None:
            dims_head[0] = in_features

        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head = nn.Sequential(*layers_list)

    def forward(self, x):
        """Forward"""
        h = self.backbone(x)
        h = self.head(h)
        return h
    
    
########
from radam import Over9000

def WrapperOver9000(param_groups,**kwargs):
    return OptimWrapper(param_groups, Over9000)

In [ ]:
class ImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class=1, in_channels=3,pretrained_path=''):
        super().__init__()
        self.model = timm.create_model(model_arch,in_chans=in_channels, pretrained=False)
        self.model=self.load_pretrain(self.model,pretrained_path)
        num_ftrs = self.model.head.in_features
        self.model.head = nn.Linear(num_ftrs, n_class)
    def load_pretrain(self,model,pretrained_path):
        checkpoint = torch.load(pretrained_path, map_location='cpu')
        print("Load ckpt from %s" % pretrained_path)
        checkpoint_model = checkpoint['model']
        state_dict = model.state_dict()
        for k in ['head.weight', 'head.bias']:
            if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
                print(f"Removing key {k} from pretrained checkpoint")
                del checkpoint_model[k]
        utils.load_state_dict(model, checkpoint_model, prefix='')
        return model
    def forward(self, x):
        x = self.model(x)
        x=x.flatten()
        return x

In [ ]:
class BasicImageModel(nn.Module):
    
    def __init__(self, 
                 base_name: str, 
                 dims_head: tp.List[int],
                 pretrained=True, 
                 in_channels: int=3, 
                 image_size: int=224,
                 drop_path_rate: float=0.1
    ):
        """Initialize"""
        self.base_name = base_name
        super().__init__()
        model_name = base_name.split("-")[0]
        assert timm.is_model(model_name), "you can use only models in timm."
        
        if model_name[:4] == "volo":
            base_model = timm.create_model(
                model_name, img_size=image_size,
                mix_token=False, 
                return_dense=False, 
                drop_path_rate=0.3, # 0.4,  0.1
            )
            in_features = base_model.head.in_features
            if pretrained:
                volo_load_weights(base_model, VOLO_CHECHPOINTS[base_name], strict=False)
            
            if in_channels != 3:
                # # change input channel
                # # I follow the manner used in timm.
                first_conv = base_model.patch_embed.conv[0]
                w_t = first_conv.weight.data  # shape: (out_ch, 3, 7, 7)
                if in_channels == 1:
                    new_w_t = w_t.sum(axis=1, keepdims=True)  # shape: (out_ch, 1, 7, 7)
                else:
                    n_repeats = (in_channels + 3 - 1) // 3
                    new_w_t = w_t.repeat((1, n_repeats, 1, 1))
                    new_w_t = new_w_t[:, :in_channels]
                    new_w_t = new_w_t * 3 / in_channels  # shape: (out_ch, in_channels, 7, 7)

                first_conv.weight.data = new_w_t
        else:
            base_model = timm.create_model(
                base_name, pretrained=pretrained, in_chans=in_channels)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)
            
        base_model.reset_classifier(num_classes=0)
        self.backbone = base_model
        print(f"{base_name}: {in_features}")
        
        # # prepare head clasifier
        if dims_head[0] is None:
            dims_head[0] = in_features

        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head = nn.Sequential(*layers_list)

    def forward(self, x):
        """Forward"""
        h = self.backbone(x)
        h = self.head(h)
        return h

In [ ]:
class VoloModel(nn.Module):
    
    def __init__(self, 
                 base_name: str, 
                 dims_head: tp.List[int],
                 pretrained=True, 
                 in_channels: int=3, 
                 image_size: int=224,
                 drop_path_rate: float=0.1
    ):
        """Initialize"""
        self.base_name = base_name
        super().__init__()
        model_name = base_name.split("-")[0]
        assert timm.is_model(model_name), "you can use only models in timm."
        
        if model_name[:4] == "volo":
            base_model = timm.create_model(
                model_name, img_size=image_size,
                mix_token=False, 
                return_dense=False, 
                drop_path_rate=0.3, # 0.4,  0.1
            )
            in_features = base_model.head.in_features
            if pretrained:
                volo_load_weights(base_model, VOLO_CHECHPOINTS[base_name], strict=False)
            
            if in_channels != 3:
                # # change input channel
                # # I follow the manner used in timm.
                first_conv = base_model.patch_embed.conv[0]
                w_t = first_conv.weight.data  # shape: (out_ch, 3, 7, 7)
                if in_channels == 1:
                    new_w_t = w_t.sum(axis=1, keepdims=True)  # shape: (out_ch, 1, 7, 7)
                else:
                    n_repeats = (in_channels + 3 - 1) // 3
                    new_w_t = w_t.repeat((1, n_repeats, 1, 1))
                    new_w_t = new_w_t[:, :in_channels]
                    new_w_t = new_w_t * 3 / in_channels  # shape: (out_ch, in_channels, 7, 7)

                first_conv.weight.data = new_w_t
        else:
            base_model = timm.create_model(
                base_name, pretrained=pretrained, in_chans=in_channels)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)
            
        base_model.reset_classifier(num_classes=0)
        self.backbone = base_model
        print(f"{base_name}: {in_features}")
        
        # # prepare head clasifier
        if dims_head[0] is None:
            dims_head[0] = in_features

        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head = nn.Sequential(*layers_list)

    def forward(self, x):
        """Forward"""
        h = self.backbone(x)
        h = self.head(h)
        return h

In [ ]:
class AlbumentationsTransform(RandTransform):
	"A transform handler for multiple `Albumentation` transforms"
	split_idx, order = None, 2

	def __init__(self, train_aug, valid_aug):
		store_attr()

	def before_call(self, b, split_idx):
		self.idx = split_idx

	def encodes(self, img: PILImage):
		if self.idx == 0:
			aug_img = self.train_aug(image=np.array(img))['image']
		else:
			aug_img = self.valid_aug(image=np.array(img))['image']
import random
import cv2
from matplotlib import pyplot as plt

import albumentations as A


def visualize(image):
	plt.figure(figsize=(10, 10))
	plt.axis('off')
	plt.imshow(image)




def train_aug():
	return A.Compose([
		A.Resize(384, 384),
		A.SomeOf([A.RandomContrast((0.6, 0.3)), A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20),
		          A.RandomBrightness(limit=[0.6, 0.3]), A.Flip(), A.Rotate(limit=45), A.Equalize(), A.Sharpen(),
		          A.Solarize(196), A.Posterize(), A.Affine(translate_percent=[0.1, 0.1])], n=3)
	])


def valid_aug():
	return A.Compose([
		A.Resize(384, 384),

	])


class AlbumentationsTransform(RandTransform):
	"A transform handler for multiple `Albumentation` transforms"
	split_idx, order = None, 2

	def __init__(self, train_aug, valid_aug):
		store_attr()

	def before_call(self, b, split_idx):
		self.idx = split_idx

	def encodes(self, img: PILImage):
		if self.idx == 0:
			aug_img = self.train_aug(image=np.array(img))['image']
		else:
			aug_img = self.valid_aug(image=np.array(img))['image']
		return PILImage.create(aug_img)

In [ ]:
#Final RMSE: 17.233747032380457
#Ensemble Weights: [0.35205359 0.18218062 0.19627086 0.27224672]
#Models: ['Ioa-384', 'Siti-224', 'Edrickv4-224', 'Volo-d2-384']
#Sum of weights: 1.002751786932252

########
# Final RMSE: 17.215003972534888
# Ensemble Weights: [0.32042124 0.15301671 0.15904355 0.19359245 0.18038215]
# Models: ['Ioa-384', 'Siti-224', 'Edrickv4-224', 'Volo-d4-448']
# Sum of weights: 1.0064561020267702

In [ ]:
### calc CV score using OOF
from sklearn.metrics import mean_squared_error

model_names = ['Ioa-384', 'Siti-224', 'Edrickv4-224', 'Volo-d2-384', 'Volo-d4-224'] # 'Volo-d4-224'
print(model_names)

oof1 = pd.read_csv('../input/exp03-swinl384-5kf-cv1746/train_with_oof_seed_365.csv')
oof1['Id'] = oof1['path'].apply(lambda s: s.split('/')[-1].split('.')[0])  
oof1 = oof1.sort_values(by='Id')
print("Model 1 RMSE (no TTA):", mean_squared_error(oof1["oof"], oof1["Pawpularity"], squared=False).round(5) )


oof2 = pd.read_csv('../input/fastai-mixup-lb01794/train_with_oof.csv')
oof2['Id'] = oof2['path'].apply(lambda s: s.split('/')[-1].split('.')[0])  
oof2 = oof2.sort_values(by='Id')
print("Model 2 RMSE (no TTA):", mean_squared_error(oof2["oof"], oof2["Pawpularity"], squared=False).round(5) )


# oof3 = pd.read_csv('../input/edrick-swin224-5kf-cv1761-lb1784/train_with_oof.csv')
oof3 = pd.read_csv('../input/convnext-oof-384/train_with_oof.csv')
oof3['Id'] = oof3['path'].apply(lambda s: s.split('/')[-1].split('.')[0])  
oof3 = oof3.sort_values(by='Id')
print("Model 3 RMSE (no TTA):", mean_squared_error(oof3["oof"], oof3["Pawpularity"], squared=False).round(5) )


# oof4 = pd.read_csv('../input/pet-swinl-224-mixup-seed-3407/train_with_oof_seed_3407.csv')
oof4 = pd.read_csv('../input/best-volo-d2-weight/volo_oof.csv')
oof4['Id'] = oof4['path'].apply(lambda s: s.split('/')[-1].split('.')[0])  
oof4 = oof4.sort_values(by='Id')
print("Model 4 RMSE (no TTA):", mean_squared_error(oof4["oof"], oof4["Pawpularity"], squared=False).round(5) )
#print("Model 4 RMSE (with TTA):", mean_squared_error(oof4["oof_tta"], oof4["Pawpularity"], squared=False).round(5) )


oof5 = pd.read_csv('../input/biet-petfinder/train_with_oof.csv')
oof5['Id'] = oof5['path'].apply(lambda s: s.split('/')[-1].split('.')[0])  
oof5 = oof5.sort_values(by='Id')
print("Model 5 RMSE (no TTA):", mean_squared_error(oof5["oof"], oof5["Pawpularity"], squared=False).round(5) )


tgt = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
tgt = tgt.sort_values(by='Id')
tgt = tgt[tgt.Id.isin(oof2.Id.tolist())]['Pawpularity']
print('-'*20)

#################
p = (oof1["oof"].values + oof2["oof"].values)/2
# p = (oof2["oof"].values + oof3["oof"].values)/2
print('\navg ens 2M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

p = 0.6 * oof1["oof"].values + 0.4 * oof2["oof"].values
# p = 0.4 * oof2["oof"].values + 0.6 * oof3["oof"].values
print('weighted ens 2M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

print('-'*20)
p = (oof1["oof"].values + oof2["oof"].values + oof3["oof"].values)/3
print('\navg ens 3M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

# p = 0.4 * oof1["oof"].values + 0.3 * oof2["oof"].values + 0.3 * oof3["oof"].values
# print('weighted ens 3M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

# p = 0.5 * oof1["oof"].values + 0.3 * oof2["oof"].values + 0.2 * oof3["oof"].values
# print('weighted ens 3M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

# p = 0.5 * oof1["oof"].values + 0.2 * oof2["oof"].values + 0.3 * oof3["oof"].values
# print('weighted ens 3M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )


print('-'*20)
p = (oof1["oof"].values + oof2["oof"].values + oof3["oof"].values + oof4["oof"].values)/4
print('\navg ens 4M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

p = 0.5 * oof1["oof"].values + 0.15 * oof2["oof"].values + 0.2 * oof3["oof"].values + 0.15 * oof4["oof"].values
print('weighted ens 4M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

p = 0.5 * oof1["oof"].values + 0.1 * oof2["oof"].values + 0.2 * oof3["oof"].values + 0.2 * oof4["oof"].values
print('weighted ens 4M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )

p = 0.342 * oof1["oof"].values + 0.1774 * oof2["oof"].values + 0.19489 * oof3["oof"].values + 0.2975 * oof4["oof"].values
print('weighted ens 4M RMSE (no TTA):', mean_squared_error(p, tgt, squared=False).round(5) )


p = 0.2 * oof1["oof"].values + 0.2 * oof2["oof"].values + 0.2 * oof3["oof"].values + 0.2 * oof4["oof"].values + 0.2 * oof5["oof"].values
print('avg ens 5M RMSE:', mean_squared_error(p, tgt, squared=False).round(5) )

p = 0.4 * oof1["oof"].values + 0.1 * oof2["oof"].values + 0.2 * oof3["oof"].values + 0.2 * oof4["oof"].values + 0.1 * oof5["oof"].values
print('weighted ens 5M RMSE:', mean_squared_error(p, tgt, squared=False).round(5) )

w = [0.28180586,0.13010029,0.28598953,0.16734793,0.18953224]
p = (w[0]*oof1["oof"].values + w[1] * oof2["oof"].values + w[2] * oof3["oof"].values + w[3] * oof4["oof"].values + w[4] * oof5["oof"].values) -2.3280542645433115

print('Ridge weights ens 5M RMSE:', mean_squared_error(p, tgt, squared=False).round(5) )


In [ ]:
w[0]

In [ ]:
import gc

In [ ]:
del tgt, oof1, oof2, oof3, oof4, oof5, p
gc.collect();

In [ ]:

# ## Edrick's LB 17.80
# def inf_run_1(fold):
#     learn = load_learner(fname=Path(f'../input/fastai-mixup/model_fold_{fold}.pkl'), cpu=False)
#     print(f'Device: {learn.dls.device} ')
#     test_dl = learn.dls.test_dl(test)
#     preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
#     return preds


## Ioa CV 17.46
def inf_run_1(fold):
    learn = load_learner(fname=Path(f'../input/exp03-swinl384-5kf-cv1746/model_fold_{fold}.pkl'), cpu=False)
    learn = learn.to_fp16()
    #print(f'Device: {learn.dls.device} ')
    test_dl = learn.dls.test_dl(test)
    #preds, _ = learn.tta(dl=test_dl, n=3, beta=0.3) ## 
    #preds, _ = learn.tta(dl=test_dl, n=5, beta=0.6)  # beta 0.6 CV 17.419
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.)
    return preds.numpy()[:,0] 



## siti's CV 17.55/17.63 -- LB 17.74
def inf_run_2(fold):
    learn = load_learner(fname=Path(f'../input/fastai-mixup-lb01794/model_fold_{fold}.pkl'), cpu=False) 
    learn = learn.to_fp16()
    #print(f'Device: {learn.dls.device} ')
    test_dl = learn.dls.test_dl(test)
    #preds, _ = learn.tta(dl=test_dl, n=5, beta=0.38) ## best n, beta 
    ##preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.)
    return preds.numpy()[:,0] 


# # ## Edrick's LB 17.84
# def inf_run_3(fold):
#     learn = load_learner(fname=Path(f'../input/edrick-swin224-5kf-cv1761-lb1784/model_fold_{fold}.pkl'), cpu=False)
#     #print(f'Device: {learn.dls.device} ')
#     test_dl = learn.dls.test_dl(test)
#     preds, _ = learn.tta(dl=test_dl, n=5, beta=0.5)  ## TODO find best n, beta 
#     return preds


# ## ConvNext CV 17.6
def inf_run_3(fold):
    if True:
        learn = load_learner(fname=Path(f'../input/convnet384/ConvNext384/model_fold_{fold}.pkl'), cpu=False)
    else:
        learn = load_learner(fname=Path(f'../input/fastai-mixup-convnextlarge-3-4/model_fold_{fold}.pkl'), cpu=False)
    learn = learn.to_fp16()
    #print(f'Device: {learn.dls.device} ')
    test_dl = learn.dls.test_dl(test)
    #preds, _ = learn.tta(dl=test_dl, n=5, beta=0.6)  ## #beta 0.6 17.257
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.) 
    return preds.numpy() 


# ## Mrigr Volo-d2 
def inf_run_4(fold):
    learn = load_learner(fname=Path(f'../input/best-volo-d2-weight/model_fold_{fold}.pkl'), cpu=False)
    learn = learn.to_fp16()
    #print(f'Device: {learn.dls.device} ')
    test_dl = learn.dls.test_dl(test)
    # preds, _ = learn.tta(dl=test_dl, n=5, beta=0.5)  ## 
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.)
    return preds.numpy()[:,0] 



# ## Ioa Volo-d4 224 CV 17.66
def inf_run_5(fold):
    learn = load_learner(fname=Path(f'../input/biet-petfinder/model_fold_{fold}.pkl'), cpu=False)
    learn = learn.to_fp16()
    #print(f'Device: {learn.dls.device} ')
    test_dl = learn.dls.test_dl(test)
    #     preds, _ = learn.tta(dl=test_dl, n=5, beta=0.1)  ## 
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.)
    return preds.numpy()[:,0] 


# # ## Mrigrenda Volo-d2 
# def inf_run_5(fold):
#     learn = load_learner(fname=Path(f'../input/volo-d2-weight/model_fold_{fold}.pkl'), cpu=False)
#     learn = learn.to_fp16()
#     #print(f'Device: {learn.dls.device} ')
#     test_dl = learn.dls.test_dl(test)
#     preds, _ = learn.tta(dl=test_dl, n=5, beta=0.4)  ## 
#     return preds.numpy()[:,0] 



# # ## Ioa Swin224 seed3407
# def inf_run_4(fold):
#     learn = load_learner(fname=Path(f'../input/pet-swinl-224-mixup-seed-3407/model_fold_{fold}.pkl'), cpu=False)
#     learn = learn.to_fp16()
#     #print(f'Device: {learn.dls.device} ')
#     test_dl = learn.dls.test_dl(test)
#     preds, _ = learn.tta(dl=test_dl, n=5, beta=0.4)  ## 
#     return preds.numpy()[:,0] 


###################################

# # ## Mithil 384 10kf CV 17.44 ?
# def inf_run_5(fold):
#     learn = load_learner(fname=Path(f'../input/fastai-384-pred/swin_large_patch4_window12_384/model_fold_{fold}.pkl'), cpu=False)
#     learn = learn.to_fp16()
#     #print(f'Device: {learn.dls.device} ')
#     test_dl = learn.dls.test_dl(test)
#     #preds, _ = learn.tta(dl=test_dl, n=5, beta=0.4)  ## 
#     preds, _ = learn.get_preds(dl=test_dl) # 
#     return preds.numpy()[:,0] 


# # ## Mithil 224 10kf CV 17.46 ?
### ......


In [ ]:
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submission['Pawpularity'] = 0

test = submission.copy()
test['path'] = test['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/test/' + x + '.jpg')
test

In [ ]:
for fold in range(5):
# for fold in [1,2,3,4]:
    
    print(f'Inference fold: {fold} ')

    test[f'preds1_{fold}'] = inf_run_1(fold) # preds.numpy()[:,0] 

In [ ]:
for fold in range(5):
# for fold in [1,2,3,4]:
    
    print(f'Inference fold: {fold} ')
    
    test[f'preds2_{fold}'] = inf_run_2(fold)

In [ ]:
for fold in range(5):
# for fold in [1,2,3,4]:
    
    print(f'Inference fold: {fold} ')
    
    test[f'preds3_{fold}'] = inf_run_3(fold) 

In [ ]:
for fold in range(5):
# for fold in [1,2,3,4]:
    
    print(f'Inference fold: {fold} ')
    
    test[f'preds4_{fold}'] = inf_run_4(fold) 

In [ ]:
for fold in range(5):
# for fold in [1,2,3,4]:
    
    print(f'Inference fold: {fold} ')
    
    test[f'preds5_{fold}'] = inf_run_5(fold) 

In [ ]:
test.head()

In [ ]:
p1 = test[['preds1_' + str(i) for i in range(5) ]].mean(axis=1)
p2 = test[['preds2_' + str(i) for i in range(5) ]].mean(axis=1)
p3 = test[['preds3_' + str(i) for i in range(5) ]].mean(axis=1)
p4 = test[['preds4_' + str(i) for i in range(5) ]].mean(axis=1)
p5 = test[['preds5_' + str(i) for i in range(5) ]].mean(axis=1)

# p1 = test[['preds1_' + str(i) for i in [1,2,3,4] ]].mean(axis=1)
# p2 = test[['preds2_' + str(i) for i in [1,2,3,4] ]].mean(axis=1)
# p3 = test[['preds3_' + str(i) for i in [1,2,3,4] ]].mean(axis=1)
# p4 = test[['preds4_' + str(i) for i in [1,2,3,4] ]].mean(axis=1)
# p5 = test[['preds5_' + str(i) for i in [1,2,3,4] ]].mean(axis=1)

assert (len(p1)==len(p2)==len(p3)==len(p4)), "Error: Check shapes!!"

In [ ]:
# w = [0.35205359 ,0.18218062, 0.19627086, 0.27224672]

w = [0.3137313,0.13160744,0.20586328,0.20454176,0.19065228]
sum(w)

In [ ]:
p1

In [ ]:
# submission['Pawpularity'] = test[['preds_' + str(i) for i in range(5)]].mean(axis=1)
# submission['Pawpularity'] = 0.5*p1 + 0.1*p2 + 0.2*p3 + 0.2*p4

submission['Pawpularity'] = w[0]*p1 + w[1]*p2 + w[2]*p3 + w[3]*p4 + w[4]*p5 

submission['Pawpularity'] = submission['Pawpularity'] * 100
submission['Pawpularity'] = submission['Pawpularity'] -2.3280542645433115
submission.to_csv('submission.csv', index=False)
submission

In [ ]:
# submission['Pawpularity'].hist(bins=50)